In [2]:
import numpy as np
import xroms
from glob import glob
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import xrft as xrft
import warnings
import sys
import xrft
sys.path.append("/homes/metogra/iufarias/FeedbackSubmeso/useful/")
import romspickle

from dask.diagnostics import ProgressBar
import dask
from dask.distributed import Client, LocalCluster

import scipy.integrate as integ
warnings.filterwarnings("ignore")

In [3]:

plt.rcParams['text.usetex'] = False
plt.rcParams.update({'font.size': 22})

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Lucida Grande']

In [4]:
# client = Client(n_workers=90,processes=False)
# client = Client(cluster)
client = Client(n_workers=90)


In [5]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://128.8.216.13:8787/status,
Dashboard: http://128.8.216.13:8787/status,Workers: 90
Total threads: 180,Total memory: 502.98 GiB
Status: running,Using processes: False
Comm: inproc://128.8.216.13/550117/1,Workers: 90
Dashboard: http://128.8.216.13:8787/status,Total threads: 180
Started: Just now,Total memory: 502.98 GiB
Comm: inproc://128.8.216.13/550117/93,Total threads: 2
Dashboard: http://128.8.216.13:39849/status,Memory: 5.59 GiB
Nanny: None,


In [5]:
# chunks={'time_counter':10, 'x_rho':50, 'y_rho':190,'s_rho':80,'s_w':80}
chunks={'time_counter':1,'x_rho':1000,"x_u":1000,"x_v":1000,
                        'y_rho':1520,"y_u":1520,"y_v":1520,'s_rho':80,'s_w':80}


path='/data/pacific/lrenault/SASI/CROCO/'

varf=xr.open_mfdataset(path + 'FULL/SASI*2012-01*.nc',data_vars='minimal',chunks=chunks)
vart=xr.open_mfdataset(path + 'SMTH/SASI*2012-01*.nc',data_vars='minimal',chunks=chunks)

# varf=xr.open_mfdataset(path + 'FULL/SASI*2012*.nc',data_vars='minimal',chunks=chunks)
# vart=xr.open_mfdataset(path + 'SMTH/SASI*2012*.nc',data_vars='minimal',chunks=chunks)

In [6]:
varx = varf.rename({"time_counter":"ocean_time","x_rho": "xi_rho","x_u":"xi_u","x_v":"xi_v","x_w":"xi_w",
                    "y_rho":"eta_rho","y_v":"eta_v","y_u":"eta_u","y_w":"eta_w","s_rho":"s_rho","s_w":"s_w"})
df,gf=xroms.roms_dataset(varx,Vtransform=varx.Vtransform.data)

vary = vart.rename({"time_counter":"ocean_time","x_rho": "xi_rho","x_u":"xi_u","x_v":"xi_v",
                    "y_rho":"eta_rho","y_v":"eta_v","y_u":"eta_u","y_w":"eta_w","s_rho":"s_rho","s_w":"s_w"})
dt,gt=xroms.roms_dataset(vary,Vtransform=vary.Vtransform.data)

In [7]:
timer_h=pd.DatetimeIndex(dt.ocean_time)
timer_d=pd.DatetimeIndex(dt.ocean_time.groupby("ocean_time.dayofyear").mean())

In [14]:
rho0=df.rho0

In [9]:
rho_full=df.rho[:,:,120:-120,120:800] +rho0
rho_smth=dt.rho[:,:,120:-120,120:800] +rho0

In [10]:
z_interp=np.linspace(-2,-100, 100)

In [15]:
z=rho_full.z_rho0.compute()

In [12]:
dec=1
# Zstar_full=xr.DataArray(coords=(df.ocean_time[::dec],
#                                 z_interp,
#                                 rho_full.eta_rho.data,
#                                 rho_full.xi_rho.data),
#                       dims=['ocean_time','z_rho','eta_rho','xi_rho'])
# Zstar_smth=xr.DataArray(coords=(dt.ocean_time[::dec],
#                                 z_interp,
#                                 rho_smth.eta_rho,
#                                 rho_smth.xi_rho),
#                       dims=['ocean_time','z_rho','eta_rho','xi_rho'])

In [16]:
z_matrix=np.zeros([z_interp.shape[0],rho_full.shape[2],rho_full.shape[3]])
z_matrix[:,:,:]=z_interp[:,None,None]

z_full_interp=rho_full[0,:,:,:].xroms.isoslice(z_interp).copy().compute()
zstar_full=rho_full[0,:,:,:].xroms.isoslice(z_interp).copy().compute()
del zstar_full.attrs['grid']


In [73]:
%%time
M='01'
# day_x=-1

for day_x in range(rho_full.ocean_time.shape[0]):
    rho_full_interp=rho_full[day_x,:,:,:].xroms.isoslice(z_interp).compute()
    z_full_interp[:]=z_matrix.copy()



    flat_rho_full=rho_full_interp.stack(z_star=("z_rho","eta_rho","xi_rho"))
    # sorted_rho_full=flat_rho_full.sortby(flat_rho_full)
    # rho_zstar_full=sorted_rho_full.unstack('z_star')

    flat_z_full=z_full_interp.stack(z_star=("z_rho","eta_rho","xi_rho"))
    sorted_z_full=flat_z_full.sortby(flat_rho_full)

    zstar_full[:]=np.array(sorted_z_full).reshape(z_full_interp.shape)

    zstar_full.to_netcdf('/data/pacific/iufarias/z_star/z_star_full_M{0}_{1:02d}.nc'.format(M,day_x))


CPU times: user 1h 10min 24s, sys: 23min 38s, total: 1h 34min 3s
Wall time: 1h 33min 45s


In [74]:
z_matrix=np.zeros([z_interp.shape[0],rho_smth.shape[2],rho_smth.shape[3]])
z_matrix[:,:,:]=z_interp[:,None,None]

z_smth_interp=rho_smth[0,:,:,:].xroms.isoslice(z_interp).copy().compute()
zstar_smth=rho_smth[0,:,:,:].xroms.isoslice(z_interp).copy().compute()
del zstar_smth.attrs['grid']


In [75]:
%%time
M='01'
# day_x=-1

for day_x in range(rho_smth.ocean_time.shape[0]):
    rho_smth_interp=rho_smth[day_x,:,:,:].xroms.isoslice(z_interp).compute()
    z_smth_interp[:]=z_matrix.copy()



    flat_rho_smth=rho_smth_interp.stack(z_star=("z_rho","eta_rho","xi_rho"))
    # sorted_rho_full=flat_rho_full.sortby(flat_rho_full)
    # rho_zstar_full=sorted_rho_full.unstack('z_star')

    flat_z_smth=z_smth_interp.stack(z_star=("z_rho","eta_rho","xi_rho"))
    sorted_z_smth=flat_z_smth.sortby(flat_rho_smth)

    zstar_smth[:]=np.array(sorted_z_smth).reshape(z_smth_interp.shape)

    zstar_smth.to_netcdf('/data/pacific/iufarias/z_star/z_star_smth_M{0}_{1:02d}.nc'.format(M,day_x))


CPU times: user 1h 8min 15s, sys: 21min 42s, total: 1h 29min 58s
Wall time: 1h 28min 31s


In [61]:
%%time
rho_full_interp=rho_full[day_x,:,:,:].xroms.isoslice(z_interp).compute()
# rho_smth_interp=rho_smth[day_x,:,:,:].xroms.isoslice(z_interp).compute()

CPU times: user 4.37 s, sys: 1 s, total: 5.37 s
Wall time: 8.69 s


In [62]:
z_matrix=np.zeros(rho_full_interp.shape)
z_matrix[:,:,:]=z_interp[:,None,None]

z_full_interp=rho_full_interp.copy()
z_full_interp[:]=z_matrix

zstar_full=rho_full_interp.copy()


In [63]:
%%time

flat_rho_full=rho_full_interp.stack(z_star=("z_rho","eta_rho","xi_rho"))
# sorted_rho_full=flat_rho_full.sortby(flat_rho_full)
# rho_zstar_full=sorted_rho_full.unstack('z_star')

flat_z_full=z_full_interp.stack(z_star=("z_rho","eta_rho","xi_rho"))
sorted_z_full=flat_z_full.sortby(flat_rho_full)

zstar_full[:]=np.array(sorted_z_full).reshape(z_full_interp.shape)


CPU times: user 21.6 s, sys: 2.78 s, total: 24.3 s
Wall time: 24.4 s
